In [ ]:
import pathlib

import pandas as pd

## Parse the NCBI BLAST outputs

1. We BLAST the query sequences in `sequences.fna` against the NCBI 16S reference database using the web tool
2. The outputs are downloaded into `ncbi16s_desc.csv` and `ncbi16s_hits.csv`

In [ ]:
data_path = pathlib.Path("../../data/processed/kchip")
blast_path = data_path / "blast"

In [ ]:
# Headers for the hits and desc files
ncbi_hits_header = "qid,Accession,pidentity,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore".split(",")
ncbi_desc_header = "Description,Scientific Name,Max Score,Total Score,Query Cover,E value,Per. ident,Acc. Len,Accession Full".split(",")

In [ ]:
ncbi_blast_hits_file = blast_path / "ncbi16s_hits.csv"
ncbi_blast_desc_file = blast_path / "ncbi16s_desc.csv"
ncbi_blast_hits = pd.read_csv(ncbi_blast_hits_file, sep=",", names=ncbi_hits_header)
ncbi_blast_desc = pd.read_csv(ncbi_blast_desc_file, sep=",", skiprows=1, names=ncbi_desc_header)

In [ ]:
# Filter the blast hits based on E-value
ncbi_blast_hits = ncbi_blast_hits[ncbi_blast_hits.evalue <= 1e-50]
ncbi_blast_hits["qid"] = [qid.split("_")[0] for qid in ncbi_blast_hits.qid]
ncbi_blast_hits

In [ ]:
# Display the number of hits for each query sequence
ncbi_blast_hits.groupby("qid")["Accession"].count()

We need to parse the accession id from the accession string in the description file

In [ ]:
ncbi_blast_desc["Accession"] = ncbi_blast_desc["Accession Full"].apply(lambda x: x.split(",")[-1].strip('")'))
ncbi_blast_desc

Finally, we join the hits and description files together

In [ ]:
ncbi_blast_results = ncbi_blast_hits.merge(ncbi_blast_desc, how="left", on="Accession")

In [ ]:
ncbi_blast_results

In [ ]:
top5_blasthits_per_qid = ncbi_blast_results.groupby("qid", as_index=False).apply(lambda grp: grp.nlargest(5, "bitscore"))

In [ ]:
top5_blasthits_per_qid.loc[:, ["qid", "Accession", "Scientific Name", "pidentity"]].head(20)

These don't match the hits from the publication:


![image](strain_source_table.png)

## Download genomes from NCBI using BioPython (Entrez)

In [ ]:
from Bio import Entrez

In [ ]:
Entrez.email = "kishored@ornl.gov"

In [ ]:
import urllib.request

def download_file(ftp_url: str, output_file: pathlib.Path):
    url = ftp_url.replace("ftp://", "http://")
    with urllib.request.urlopen(url) as url_id:
        with open(output_file, "wb") as fid:
            fid.write(url_id.read())

def download_genomes(scientific_name: str, output_folder: pathlib.Path) -> None:
    # Get list of reference or representative sequences for the given organimsm using its name
    term = f"({scientific_name}[ORGN]) AND (reference_genome[filter] OR representative_genome[filter])"
    search_handle = Entrez.esearch(db="assembly", idtype="acc", term=term, retmax=100)
    search_record = Entrez.read(search_handle)
    # Get the the accession number of the first result
    if len(search_record["IdList"]) == 0: # type: ignore
        raise ValueError(f"No reference or representative genomes found for {scientific_name}")
    if len(search_record["IdList"]) > 1: # type: ignore
        print(f"More than one reference or representative genomes found for {scientific_name}. Using the first one.")
    assembly_acc = search_record["IdList"][0] # type: ignore
    # Use document summary to get the ftp path for the assembly
    summary_handle = Entrez.esummary(db="assembly", id=assembly_acc)
    document_summary = Entrez.read(summary_handle)
    url = document_summary['DocumentSummarySet']['DocumentSummary'][0]['FtpPath_RefSeq'] # type: ignore
    file_id = url.split("/")[-1]
    fna_url = f"{url}/{file_id}_genomic.fna.gz"
    fna_file = output_folder / f"{file_id}.fna.gz"
    download_file(fna_url, fna_file)
    gff_url = f"{url}/{file_id}_genomic.gff.gz"
    gff_file = output_folder / f"{file_id}.gff.gz"
    download_file(gff_url, gff_file)

In [ ]:
seq_folder = pathlib.Path("../../data/processed/kchip/sequences/")
for i, row in top5_blasthits_per_qid.iterrows():
    qid = row["qid"]
    scientific_name = " ".join(row["Scientific Name"].split(" ")[:2])
    print(f"{i}: Downloading fna and gff files for {scientific_name} matching {qid}")
    output_folder = seq_folder / f"{qid}"
    output_folder.mkdir(parents=True, exist_ok=True)
    download_genomes(scientific_name, output_folder=output_folder)